In [1]:
from sklearn import preprocessing
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import postgresql
import pickle
import pandas as pd

## 1. Create a DataFrame with the Census Data from HW2

Find two examples in the course content in Blackboard  
Database Details:   
user='new_haven_ds_read'  
password='new_haven_ds_secret_99'  
host='nhds.cwroivw0q1rc.us-east-1.rds.amazonaws.com'  
database='nhds'  
  
  
 SQL: sql = 'select * from nhds.uci_adults'

In [2]:
db = postgresql.open("pq://new_haven_ds_read:new_haven_ds_secret_99@nhds.cwroivw0q1rc.us-east-1.rds.amazonaws.com/nhds")
sql = 'select * from nhds.uci_adults'
ps = db.prepare(sql)
res = ps()

In [3]:
df = pd.DataFrame(res, columns=ps.column_names)
df

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,2
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,3
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,4
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,32557
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,32558
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,32559
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,32560


In [4]:
df.to_pickle('df.pkl')

In [5]:
del(df)
df

NameError: name 'df' is not defined

In [6]:
df = pd.read_pickle('df.pkl')
df

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,2
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,3
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,4
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,32557
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,32558
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,32559
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,32560


## 2. Split the data into training and testing sets

In [7]:
df = df.dropna()
for column in df.columns:
    if df[column].dtype==type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column]=le.fit_transform(df[column])

In [8]:
x = df.drop(columns=['income'])
y = df['income']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 100)

## 3. Use sklearn's LabelEncoder to make the columns with strings usuable

In [9]:
# Change of datatype
df = df.dropna()
for column in df.columns:
    if df[column].dtype==type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column]=le.fit_transform(df[column])

## 4. Create a Decision Tree with the Census Data

In [10]:
dtree = DecisionTreeClassifier(random_state = 100)
dtree.fit(x_train, y_train)

DecisionTreeClassifier(random_state=100)

In [11]:
dtree.predict(x_test)

array([0, 0, 0, ..., 1, 0, 1])

In [12]:
pred_df = pd.DataFrame(dtree.predict(x_test), columns=['pred'])
pred_df['actual'] = y_test.values
pred_df['residual'] = (pred_df['pred']-pred_df['actual']).abs()
pred_df

,pred,actual,residual
0,0,0,0
1,0,0,0
2,0,1,1
3,0,0,0
4,0,0,0
...,...,...,...
8136,0,0,0
8137,1,0,1
8138,1,1,0
8139,0,0,0


In [13]:
#Accuracy
print('Acuracy:{:.2f}'.format(dtree.score(x_test,y_test)))

Acuracy:0.81


## 5. Create a Random Forest with the same Census Data

In [14]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)

RandomForestClassifier()

In [15]:
pred_df = pd.DataFrame(rf.predict(x_test), columns=['pred'])
pred_df['actual'] = y_test.values
pred_df['residual'] = (pred_df['pred']-pred_df['actual']).abs()
pred_df

,pred,actual,residual
0,0,0,0
1,0,0,0
2,0,1,1
3,0,0,0
4,0,0,0
...,...,...,...
8136,0,0,0
8137,0,0,0
8138,1,1,0
8139,0,0,0


In [16]:
rf.predict(x_test)

array([0, 0, 0, ..., 1, 0, 1])

In [17]:
#Accuracy
print('Acuracy:{:.2f}'.format(rf.score(x_test,y_test)))

Acuracy:0.86


# 6. Compare the Accuracy of the Random Forest and the base Decision Tree.  Which is Better?  Why?

In [2]:
'We can see that the accuracy of the Random Forest of 0.86 is better than that of the Decision Tree of 0.81'

'We can see that the accuracy of the Random Forest of 0.86 is better than that of the Decision Tree of 0.81'